# Breve análise dos resultados obtidos

Neste notebook, são computados as média e mediana dos projetos coletados, dados os valores resultantes da detecção de padrões.

In [7]:
collected_projects_dir = '/home/gabriel/Personal/Study/2021.1/TG/collected-projects'

In [11]:
projects_pattern = f'{collected_projects_dir}/*/'

dirnames = !echo $projects_pattern

dirnames = dirnames[0].split(' ')
dirnames = [dirname[:-1] for dirname in dirnames] # Removendo barra restante de cada nome de diretório

dirnames

['/home/gabriel/Personal/Study/2021.1/TG/collected-projects/*']